# 第十一章 標準程式庫 (Standard Library)

## 章節概述

Go 的標準程式庫是語言的重要組成部分，提供了豐富且高品質的功能模組。本章將深入探討最常用且重要的標準程式庫套件，包括 I/O 操作、時間處理、JSON 編碼，以及 HTTP 服務開發。

### 本章重點內容
- **io 套件家族**: Reader, Writer, Closer 介面的使用
- **time 套件**: 時間處理、格式化、計時器
- **encoding/json**: JSON 資料的序列化與反序列化
- **net/http**: HTTP 客戶端與伺服器開發

### 標準程式庫的設計哲學
Go 標準程式庫遵循以下原則：
1. **介面導向**: 大量使用小而專一的介面
2. **組合優於繼承**: 透過組合實現複雜功能
3. **零值可用**: 大多數型別的零值就是可用的
4. **簡單一致**: API 設計簡潔且一致

In [ ]:
// Go 標準程式庫綜合示範
package main

import (
    "bytes"
    "encoding/json"
    "fmt"
    "io"
    "net/http"
    "os"
    "strings"
    "time"
)

// 示範結構：用戶資料
type User struct {
    ID        int       `json:"id"`
    Name      string    `json:"name"`
    Email     string    `json:"email"`
    CreatedAt time.Time `json:"created_at"`
    Active    bool      `json:"active"`
}

func main() {
    fmt.Println("=== Go 標準程式庫綜合示範 ===")
    
    // 1. io 套件示範
    fmt.Println("\n1. IO 操作示範:")
    demonstrateIO()
    
    // 2. time 套件示範
    fmt.Println("\n2. 時間處理示範:")
    demonstrateTime()
    
    // 3. JSON 處理示範
    fmt.Println("\n3. JSON 處理示範:")
    demonstrateJSON()
    
    // 4. HTTP 客戶端示範
    fmt.Println("\n4. HTTP 客戶端示範:")
    demonstrateHTTPClient()
}

// 示範 io 套件的使用
func demonstrateIO() {
    // 1. strings.Reader - 實現 io.Reader 介面
    text := "Hello, Go standard library!"
    reader := strings.NewReader(text)
    
    fmt.Printf("  原始文字: %s\n", text)
    fmt.Printf("  Reader 長度: %d\n", reader.Len())
    
    // 讀取前 10 個字節
    buffer := make([]byte, 10)
    n, err := reader.Read(buffer)
    if err != nil {
        fmt.Printf("  讀取錯誤: %v\n", err)
        return
    }
    fmt.Printf("  讀取了 %d 字節: %s\n", n, string(buffer[:n]))
    
    // 2. bytes.Buffer - 實現 io.Reader 和 io.Writer
    var buf bytes.Buffer
    
    // 寫入資料
    buf.WriteString("這是 ")
    buf.WriteString("bytes.Buffer ")
    buf.WriteString("的示範")
    
    fmt.Printf("  Buffer 內容: %s\n", buf.String())
    fmt.Printf("  Buffer 長度: %d\n", buf.Len())
    
    // 3. io.Copy - 複製資料
    source := strings.NewReader("要複製的資料")
    var destination bytes.Buffer
    
    copied, err := io.Copy(&destination, source)
    if err != nil {
        fmt.Printf("  複製錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("  複製了 %d 字節: %s\n", copied, destination.String())
    
    // 4. io.MultiReader - 組合多個 Reader
    reader1 := strings.NewReader("第一部分 ")
    reader2 := strings.NewReader("第二部分 ")
    reader3 := strings.NewReader("第三部分")
    
    multiReader := io.MultiReader(reader1, reader2, reader3)
    
    var result bytes.Buffer
    io.Copy(&result, multiReader)
    
    fmt.Printf("  多重讀取結果: %s\n", result.String())
}

// 示範 time 套件的使用
func demonstrateTime() {
    // 1. 基本時間操作
    now := time.Now()
    fmt.Printf("  目前時間: %s\n", now.Format(time.RFC3339))
    
    // 2. 時間格式化（Go 的特殊格式）
    customFormat := "2006-01-02 15:04:05"
    fmt.Printf("  自訂格式: %s\n", now.Format(customFormat))
    
    // 3. 時間解析
    timeStr := "2024-01-15 10:30:00"
    parsed, err := time.Parse(customFormat, timeStr)
    if err != nil {
        fmt.Printf("  解析錯誤: %v\n", err)
    } else {
        fmt.Printf("  解析時間: %s\n", parsed.Format(time.RFC3339))
    }
    
    // 4. 時間計算
    future := now.Add(24 * time.Hour) // 加 24 小時
    duration := future.Sub(now)        // 計算時間差
    
    fmt.Printf("  24小時後: %s\n", future.Format(customFormat))
    fmt.Printf("  時間差: %v\n", duration)
    
    // 5. 計時器示範
    fmt.Println("  計時器測試開始...")
    start := time.Now()
    
    // 模擬一些工作
    time.Sleep(100 * time.Millisecond)
    
    elapsed := time.Since(start)
    fmt.Printf("  工作耗時: %v\n", elapsed)
    
    // 6. 時區處理
    utc := now.UTC()
    fmt.Printf("  UTC 時間: %s\n", utc.Format(customFormat))
    
    // 載入特定時區（如果可用）
    if location, err := time.LoadLocation("Asia/Taipei"); err == nil {
        taipei := now.In(location)
        fmt.Printf("  台北時間: %s\n", taipei.Format(customFormat))
    }
}

// 示範 JSON 處理
func demonstrateJSON() {
    // 1. 建立測試資料
    users := []User{
        {
            ID:        1,
            Name:      "張三",
            Email:     "zhang@example.com",
            CreatedAt: time.Now(),
            Active:    true,
        },
        {
            ID:        2,
            Name:      "李四",
            Email:     "li@example.com",
            CreatedAt: time.Now().Add(-24 * time.Hour),
            Active:    false,
        },
    }
    
    // 2. Marshal - 結構轉 JSON
    jsonData, err := json.Marshal(users)
    if err != nil {
        fmt.Printf("  JSON Marshal 錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("  JSON 資料: %s\n", string(jsonData))
    
    // 3. MarshalIndent - 格式化的 JSON
    prettyJSON, err := json.MarshalIndent(users, "", "  ")
    if err != nil {
        fmt.Printf("  格式化錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("  格式化 JSON:\n%s\n", string(prettyJSON))
    
    // 4. Unmarshal - JSON 轉結構
    var parsedUsers []User
    err = json.Unmarshal(jsonData, &parsedUsers)
    if err != nil {
        fmt.Printf("  JSON Unmarshal 錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("  解析結果: 找到 %d 個用戶\n", len(parsedUsers))
    for _, user := range parsedUsers {
        fmt.Printf("    用戶: %s (%s) - 狀態: %t\n", 
            user.Name, user.Email, user.Active)
    }
    
    // 5. 處理動態 JSON - 使用 map[string]interface{}
    dynamicJSON := `{"message":"Hello","code":200,"data":[1,2,3]}`
    
    var dynamic map[string]interface{}
    err = json.Unmarshal([]byte(dynamicJSON), &dynamic)
    if err != nil {
        fmt.Printf("  動態 JSON 解析錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("  動態 JSON 解析:\n")
    for key, value := range dynamic {
        fmt.Printf("    %s: %v (型別: %T)\n", key, value, value)
    }
}

// 示範 HTTP 客戶端
func demonstrateHTTPClient() {
    // 1. 簡單的 GET 請求
    fmt.Println("  發送 GET 請求到 httpbin.org...")
    
    resp, err := http.Get("https://httpbin.org/json")
    if err != nil {
        fmt.Printf("  GET 請求錯誤: %v\n", err)
        return
    }
    defer resp.Body.Close()
    
    fmt.Printf("  回應狀態: %s\n", resp.Status)
    fmt.Printf("  回應標頭: Content-Type = %s\n", 
        resp.Header.Get("Content-Type"))
    
    // 讀取回應內容
    body, err := io.ReadAll(resp.Body)
    if err != nil {
        fmt.Printf("  讀取回應錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("  回應長度: %d 字節\n", len(body))
    
    // 解析 JSON 回應
    var jsonResp map[string]interface{}
    if err := json.Unmarshal(body, &jsonResp); err == nil {
        fmt.Printf("  JSON 回應包含 %d 個欄位\n", len(jsonResp))
    }
    
    // 2. 建立自訂的 HTTP 客戶端
    fmt.Println("\n  使用自訂 HTTP 客戶端...")
    
    client := &http.Client{
        Timeout: 10 * time.Second, // 設定超時時間
    }
    
    // 建立請求
    req, err := http.NewRequest("GET", "https://httpbin.org/headers", nil)
    if err != nil {
        fmt.Printf("  建立請求錯誤: %v\n", err)
        return
    }
    
    // 設定請求標頭
    req.Header.Set("User-Agent", "Go-HTTP-Client/1.0")
    req.Header.Set("Accept", "application/json")
    
    // 發送請求
    resp2, err := client.Do(req)
    if err != nil {
        fmt.Printf("  自訂請求錯誤: %v\n", err)
        return
    }
    defer resp2.Body.Close()
    
    fmt.Printf("  自訂請求狀態: %s\n", resp2.Status)
    
    // 簡單顯示回應內容
    body2, _ := io.ReadAll(resp2.Body)
    if len(body2) > 200 {
        fmt.Printf("  回應內容預覽: %s...\n", string(body2[:200]))
    } else {
        fmt.Printf("  回應內容: %s\n", string(body2))
    }
}